# Gemma and QWEN7B

This notebook uses two Models, Qwen7B and Gemma, to classify the posts.

It will then ensemble the two predictions. I don't know how to best ensemble the two models yet, for now we'll average.


Credits: Kudos to the authors of these notebooks. Thanks for sharing:
- https://www.kaggle.com/code/yangyefd/batch-gemma3-sample-rules-classification
- https://www.kaggle.com/code/aerdem4/jigsaw-acrc-qwen7b-finetune-logits-processor-zoo

## How to install dependencies

Interactive notebooks: Click on Add-Ons->Install Dependencies, then Run.

Submission: Dependencies will be installed automatically

## Prepare Qwen7b and Gemma

In [47]:
import vllm, torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor


In [ ]:
def predict_with_model (sys_prompt, prompts, model_path):
    model = vllm.LLM(
        model_path,
        tensor_parallel_size=torch.cuda.device_count(),
        gpu_memory_utilization=0.45,
        trust_remote_code=True,
        dtype="half",
        enforce_eager=True,
        max_model_len=4096,
        disable_log_stats=True,
        enable_prefix_caching=True
    )
    tokenizer = model.get_tokenizer()

In [ ]:


qwen7b_model_path = "/kaggle/input/jigsaw-acrc-qwen7b-v01"

qwen7b = vllm.LLM(
    qwen7b_model_path,
    tensor_parallel_size=torch.cuda.device_count(),
    gpu_memory_utilization=0.45,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=4096,
    disable_log_stats=True,
    enable_prefix_caching=True
)
qwen7b_tokenizer = qwen7b.get_tokenizer()

INFO 07-25 10:43:24 [config.py:1604] Using max model len 4096
WARNING 07-25 10:43:24 [cuda.py:103] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-25 10:43:24 [llm_engine.py:228] Initializing a V0 LLM engine (v0.10.0) with config: model='/kaggle/input/jigsaw-acrc-qwen7b-v01', speculative_config=None, tokenizer='/kaggle/input/jigsaw-acrc-qwen7b-v01', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=2, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), obs

2025-07-25 10:43:29.360254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753440209.382065     388 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753440209.388843     388 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 07-25 10:43:34 [__init__.py:235] Automatically detected platform cuda.
(VllmWorkerProcess pid=388) INFO 07-25 10:43:35 [multiproc_worker_utils.py:226] Worker ready; awaiting tasks
(VllmWorkerProcess pid=388) INFO 07-25 10:43:36 [cuda.py:346] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
(VllmWorkerProcess pid=388) INFO 07-25 10:43:36 [cuda.py:395] Using XFormers backend.
INFO 07-25 10:43:37 [multiproc_worker_utils.py:138] Terminating local vLLM worker processes


[W725 10:43:48.927688340 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:43:58.391046359 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:44:09.997048013 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:44:10.950126792 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:44:20.927328095 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:44:32.354583793 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:44:46.921594314 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:45:01.145451603 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:45:14.797038094 socket.cpp:200] [c10d] The hostname of the client socket cannot be retrieved. err=-3
[W725 10:4

!ls /kaggle/input/

In [ ]:
import vllm, torch
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor


GEMMA_PATH = "/kaggle/input/gemma-7b-gguf/gemma-2-9b-it.Q5_K_M.gguf"

gemma = vllm.LLM(
    GEMMA_PATH,
    tensor_parallel_size=torch.cuda.device_count(),
    gpu_memory_utilization=0.45,
    trust_remote_code=True,
    dtype="half",
    enforce_eager=True,
    max_model_len=4096,
    disable_log_stats=True,
    enable_prefix_caching=True
)
gemma_tokenizer = gemma.get_tokenizer()


## Prepare Training data and Examples 

In [ ]:
import os, sys
import pandas as pd

data_path = "/kaggle/input/jigsaw-agile-community-rules/test.csv" \
                if os.getenv('KAGGLE_IS_COMPETITION_RERUN') \
                else "/kaggle/input/jigsaw-agile-community-rules/train.csv"

df = pd.read_csv(data_path)
df.head(10)

## Preparing Prompts

In [ ]:
SYS_PROMPT = """
You are given a comment on reddit. Your task is to classify if it violates the given rule. Only respond Yes/No.
"""

qwen7b_prompts = []
gemma_prompts = []
for i, row in df.iterrows():
    text = f"""
r/{row.subreddit}
Rule: {row.rule}

1) {row.positive_example_1}
Violation: Yes

2) {row.negative_example_1}
Violation: No

3) {row.negative_example_2}
Violation: No

4) {row.positive_example_2}
Violation: Yes

5) {row.body}
"""
    print(text)
    
    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": text}
    ]

    qwen_prompt = qwen7b_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    ) + "Violation: "
    qwen7b_prompts.append(qwen_prompt)
    gemma_prompt = gemma_tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    ) + "Violation: "
    qwen7b_prompts.append(gemma_prompt)



## Predict / Apply MultiChoice processor

In [ ]:
qwen_mclp = MultipleChoiceLogitsProcessor(qwen7b_tokenizer, 
                                     choices=["Yes", "No"])

qwen7b_outputs = qwen7b_llm.generate(
    prompts,
    vllm.SamplingParams(
        seed=0,
        skip_special_tokens=True,
        max_tokens=1,
        logits_processors=[mclp],
        logprobs=len(mclp.choices)

    ),
    use_tqdm=True
)


qwen7b_logprobs = []
for lps in [output.outputs[0].logprobs[0].values() for output in qwen7b_outputs]:
    qwen7b_logprobs.append({lp.decoded_token: lp.logprob for lp in list(lps)})

In [ ]:
gemma_mclp = MultipleChoiceLogitsProcessor(gemma_tokenizer, 
                                     choices=["Yes", "No"])

gemma_outputs = gemma_llm.generate(
    prompts,
    vllm.SamplingParams(
        seed=0,
        skip_special_tokens=True,
        max_tokens=1,
        logits_processors=[mclp],
        logprobs=len(mclp.choices)

    ),
    use_tqdm=True
)


gemma_logprobs = []
for lps in [output.outputs[0].logprobs[0].values() for output in gemma_outputs]:
    gemma_logprobs.append({lp.decoded_token: lp.logprob for lp in list(lps)})

## Averaging predictions and creating a submission file


In [ ]:
# Combine logprobs from Qwen and Gemma using logprob averaging
final_preds = []
for q_lp, g_lp in zip(qwen7b_logprobs, gemma_logprobs):
    avg_yes = (q_lp.get("Yes", -999) + g_lp.get("Yes", -999)) / 2
    avg_no = (q_lp.get("No", -999) + g_lp.get("No", -999)) / 2
    final_preds.append("Yes" if avg_yes > avg_no else "No")

# Format for Kaggle submission
submission_df = pd.DataFrame({
    "example_id": df["example_id"],
    "label": final_preds
})

submission_df.to_csv("submission.csv", index=False)
print("Saved submission.csv")


In [ ]:
import matplotlib.pyplot as plt

# Extract individual logprobs for plotting
qwen_yes = [lp.get("Yes", -999) for lp in qwen7b_logprobs]
qwen_no  = [lp.get("No", -999) for lp in qwen7b_logprobs]

gemma_yes = [lp.get("Yes", -999) for lp in gemma_logprobs]
gemma_no  = [lp.get("No", -999) for lp in gemma_logprobs]

avg_yes = [(qy + gy) / 2 for qy, gy in zip(qwen_yes, gemma_yes)]
avg_no  = [(qn + gn) / 2 for qn, gn in zip(qwen_no, gemma_no)]

# Plot histograms
plt.figure(figsize=(12, 6))
plt.hist(qwen_yes, bins=30, alpha=0.5, label='Qwen7B - Yes', density=True)
plt.hist(qwen_no, bins=30, alpha=0.5, label='Qwen7B - No', density=True)
plt.hist(gemma_yes, bins=30, alpha=0.5, label='Gemma - Yes', density=True)
plt.hist(gemma_no, bins=30, alpha=0.5, label='Gemma - No', density=True)
plt.hist(avg_yes, bins=30, alpha=0.6, label='Avg - Yes', histtype='step', linewidth=2)
plt.hist(avg_no, bins=30, alpha=0.6, label='Avg - No', histtype='step', linewidth=2)

plt.title("Histogram of Logprobs for 'Yes' and 'No' Predictions")
plt.xlabel("Log Probability")
plt.ylabel("Density")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


INFO 07-25 10:42:10 [multiproc_worker_utils.py:125] Killing local vLLM worker processes
